Initialization

Just loading packages and connecting to the database. 

In [3]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True


C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Creating initial table to compare row counts, member ID counts, and claim ID counts between the data warehouse and the raw data.  

In [73]:
tableCreateQuery = ''' 
DROP TABLE IF EXISTS qa_reporting.dw_iqva_claim_icd_proc_counts;

CREATE TABLE qa_reporting.dw_iqva_claim_icd_proc_counts
(
    calendar_year INT,
    dw_row_count BIGINT,
    src_row_count BIGINT,
    row_count_diff BIGINT,
    row_count_percentage FLOAT,
    dw_uth_clm_id_count BIGINT,
    dw_src_clm_id_count BIGINT,
    src_clm_count BIGINT,
    clm_count_diff BIGINT,
    clm_count_percentage FLOAT,
    dw_uth_mbr_id_count BIGINT,
    dw_src_mbr_id_count BIGINT,
    src_mbr_count BIGINT,
    mbr_count_diff BIGINT,
    mbr_count_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

In [74]:
with connection.cursor() as cursor:
    tableInsertQuery = f'''
        INSERT INTO qa_reporting.dw_iqva_claim_icd_proc_counts
        (calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
        SELECT
            DISTINCT year, 
            COUNT(a.*), 
            COUNT(DISTINCT uth_claim_id), 
            current_date
        FROM (
            SELECT 
                year, 
                uth_claim_id, 
                uth_member_id, 
                proc_cd, 
                icd_version
            FROM dw_staging.iqva_claim_icd_proc
            WHERE proc_position = 1
        ) a
        GROUP BY 1
    '''
    
    cursor.execute(tableInsertQuery)

    srcClmQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_icd_proc_counts b
        SET dw_src_clm_id_count = count
        FROM (
            SELECT year, COUNT(DISTINCT claim_id_src) AS count 
            FROM dw_staging.iqva_claim_icd_proc
            WHERE proc_position = 1
        GROUP BY 1) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(srcClmQuery)

    dwMbrQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_icd_proc_counts b
        SET dw_uth_mbr_id_count = count
        FROM (
            SELECT year, COUNT(DISTINCT uth_member_id) AS count 
            FROM dw_staging.iqva_claim_icd_proc
            WHERE proc_position = 1
        GROUP BY 1) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(dwMbrQuery)
    
    srcMbrQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_icd_proc_counts b
        SET dw_src_mbr_id_count = count
        FROM (
            SELECT year, COUNT(DISTINCT member_id_src) AS count 
            FROM dw_staging.iqva_claim_icd_proc
            WHERE proc_position = 1
        GROUP BY 1) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(srcMbrQuery)

In [78]:

with connection.cursor() as cursor: 

    compUpdateQuery = '''
        WITH clms AS (
            SELECT 
                min(year) as year, 
                pat_id, 
                derv_claimno
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
            WHERE icdprc1 IS NOT NULL AND (new_rectype != 'P' OR new_rectype IS NULL)
            GROUP BY pat_id, derv_claimno, icdprc1, icdprc2, icdprc3, icdprc4, icdprc5,
		    icdprc6, icdprc7, icdprc8, icdprc9, icdprc10,
		    icdprc11, icdprc12, diagprc_ind
        ),
        clm_counts AS (
            SELECT 
                year, 
                COUNT(*) src_row_count, 
                COUNT(DISTINCT pat_id) src_mbr_count, 
                COUNT(DISTINCT derv_claimno) src_clm_count
            FROM clms
            GROUP BY 1
        )
        update qa_reporting.dw_iqva_claim_icd_proc_counts a
        set src_row_count = b.src_row_count,
            row_count_diff = a.dw_row_count - b.src_row_count,
            row_count_percentage = 100. * abs(a.dw_row_count - b.src_row_count) / b.src_row_count,
            src_clm_count = b.src_clm_count,
            clm_count_diff = a.dw_uth_clm_id_count - b.src_clm_count,
            clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.src_clm_count) / b.src_clm_count,
            src_mbr_count = b.src_mbr_count,
            mbr_count_diff = a.dw_uth_mbr_id_count - b.src_mbr_count,   
            mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.src_mbr_count) / b.src_mbr_count
        from clm_counts b
        where a.calendar_year = b.year;
    '''

    cursor.execute(compUpdateQuery)

In [79]:
countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_icd_proc_counts;', con=connection)
countsDf.sort_values(['clm_count_percentage'], ascending=False)
print(countsDf)

    calendar_year  dw_row_count  src_row_count  row_count_diff  \
0            2018        382966         382966               0   
1            2020        394416         394417              -1   
2            2008       2895187        2899553           -4366   
3            2021        434997         434997               0   
4            2014       2116391        2116703            -312   
5            2009       2880479        2883684           -3205   
6            2019        418608         418608               0   
7            2006       1161430        1161954            -524   
8            2011       2729082        2731896           -2814   
9            2012       2252433        2253551           -1118   
10           2022        431325         431325               0   
11           2017        576196         576394            -198   
12           2013       2654123        2654471            -348   
13           2016        795903         796152            -249   
14        

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_8816\3991394403.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_icd_proc_counts;', con=connection)


ICD Procedure Codes

In [ ]:
Getting counts for different types of ICD procedure codes. 

In [56]:
icdProcQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_icd_proc_counts;
    
    SELECT 
        year, 
        proc_cd, 
        COUNT(*) AS proc_count
    INTO qa_reporting.dw_iqva_icd_proc_counts
    FROM dw_staging.iqva_claim_icd_proc
    GROUP BY 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(icdProcQuery)

In [57]:
icdProcDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_icd_proc_counts;', con=connection)
print(icdProcDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_8816\1029596272.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  icdProcDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_icd_proc_counts;', con=connection)


        year  proc_cd  proc_count
0       2015  04HD3DZ           1
1       2009     7906        1351
2       2016  0MBK0ZZ           1
3       2020  BD13YZZ          19
4       2022  0LUW07Z           1
...      ...      ...         ...
195092  2022  0SCC4ZZ           7
195093  2022  10907ZU           7
195094  2020  3E05303           1
195095  2022  02713E6           1
195096  2019  0JB80ZX          29

[195097 rows x 3 columns]


In [58]:
icdProcAntiJoinQuery = '''
    SELECT a.*
    FROM qa_reporting.dw_iqva_icd_proc_counts a
    LEFT JOIN reference_tables.ref_cms_icd_pcs_codes b ON a.proc_cd = cd_value
    WHERE b.cd_value IS NULL;
'''

invalidProcDf = pd.read_sql(icdProcAntiJoinQuery, con=connection)
print(invalidProcDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_8816\3751610776.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalidProcDf = pd.read_sql(icdProcAntiJoinQuery, con=connection)


       year proc_cd  proc_count
0      2021   99347           3
1      2012   45990           1
2      2007    2963          28
3      2008   67105           7
4      2008    2963          13
...     ...     ...         ...
13977  2009   A4208           2
13978  2007     946           2
13979  2012     389           2
13980  2008     389           6
13981  2020   49451           1

[13982 rows x 3 columns]


In [59]:
invalidProcDf.groupby('year')['proc_count'].sum()

year
2006    12215
2007    14572
2008    59787
2009    33771
2010    33693
2011    32230
2012    19301
2013    13057
2014     8477
2015     5574
2016     2004
2017     1591
2018     1295
2019      768
2020     5251
2021     4023
2022      236
2023      129
Name: proc_count, dtype: int64

In [60]:
procCompDf = pd.DataFrame(
    {
        'overall_proc_count': icdProcDf.groupby('year')['proc_count'].sum(),
        'invalid_proc_count': invalidProcDf.groupby('year')['proc_count'].sum(),
        'valid_proc_count': icdProcDf.groupby('year')['proc_count'].sum() - invalidProcDf.groupby('year')['proc_count'].sum()
    }
)
procCompDf.loc[procCompDf['valid_proc_count'].isna(),'valid_proc_count'] = procCompDf.loc[procCompDf['valid_proc_count'].isna(),'overall_proc_count']
procCompDf['valid_proc_count'] =  procCompDf['valid_proc_count'].astype(int)
procCompDf['invalid_to_valid_percent'] = 100. * procCompDf['invalid_proc_count'] / procCompDf['valid_proc_count']
procCompDf

,overall_proc_count,invalid_proc_count,valid_proc_count,invalid_to_valid_percent
year,,,,
2006,2195397,12215,2183182,0.559504
2007,2056248,14572,2041676,0.713727
2008,4953542,59787,4893755,1.221700
2009,4833697,33771,4799926,0.703573
2010,4414340,33693,4380647,0.769133
2011,4544803,32230,4512573,0.714227
2012,3889884,19301,3870583,0.498659
2013,4284357,13057,4271300,0.305691
2014,3520839,8477,3512362,0.241348


In [61]:
invalidProcDf.groupby('year').max()

,proc_cd,proc_count
year,,
2006,V571,8978
2007,V2797,11790
2008,V761,11145
2009,V578,11618
2010,V586,11098
2011,~,10111
2012,~,9350
2013,~,8746
2014,~,7618


Procedure Code Position

Getting counts for the position that procedure codes are occurring in. 

In [62]:
procCdeQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_proc_position;

    SELECT 
        year, 
        proc_position, 
        COUNT(DISTINCT uth_claim_id) AS claim_count
    INTO qa_reporting.dw_iqva_proc_position
    FROM dw_staging.iqva_claim_icd_proc
    GROUP BY 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(procCdeQuery)

In [63]:
procCdeDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_proc_position;', con=connection)
procCdeDf.sort_values(['year', 'proc_position'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_8816\429261577.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  procCdeDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_proc_position;', con=connection)


,year,proc_position,claim_count
77,2006,1,1146293
55,2006,2,512891
164,2006,3,255587
172,2006,4,132609
108,2006,5,81541
...,...,...,...
90,2023,8,8242
104,2023,9,109
16,2023,10,73
24,2023,11,52


In [65]:
procCdeDf['proc_position'].unique()

array([ 4,  3, 10, 12,  7,  8,  9,  6, 11,  1,  2,  5], dtype=int64)

In [66]:
procCdeDf['row_rank'] = procCdeDf.sort_values(['year', 'claim_count'], ascending=[True, False]).groupby(['year']).cumcount()+1
procCdeDf['position_check'] = procCdeDf['row_rank'] == procCdeDf['proc_position']
procCdeDf[~procCdeDf['position_check']]

,year,proc_position,claim_count,row_rank,position_check


ICD Version

In [ ]:
Comparing counts for different ICD versions.

In [71]:
icdVersQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_icd_proc_version_count;
    SELECT 
        year, 
        icd_version, 
        COUNT(DISTINCT uth_claim_id) AS claim_count
    INTO qa_reporting.dw_iqva_icd_proc_version_count
    FROM dw_staging.iqva_claim_icd_proc
    GROUP BY 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(icdVersQuery)

icdVersDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_icd_proc_version_count WHERE icd_version IS NOT NULL;', con=connection)
icdVersDf.sort_values(['year', 'icd_version'])
print(icdVersDf)

    year icd_version  claim_count
0   2021           0       436717
1   2015           9       628606
2   2007           9      1025723
3   2010           9      2629680
4   2016           9            5
5   2022           9            1
6   2022           0       433126
7   2009           9      2890012
8   2006           9      1151378
9   2009           0            1
10  2023           0       137832
11  2016           0       793086
12  2012           9      2235522
13  2015           0       220144
14  2013           9      2652461
15  2018           0       383002
16  2011           9      2718396
17  2014           9      2113874
18  2020           0       396674
19  2019           0       418690
20  2017           0       575276
21  2008           9      2907991


C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_8816\3168425912.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  icdVersDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_icd_proc_version_count WHERE icd_version IS NOT NULL;', con=connection)
